In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../data/hong.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


# def map_docs(inputs):
#     documents = inputs["documents"]
#     question = inputs["question"]
#     return "\n\n".join(
#         map_doc_chain.invoke(
#             {"context": doc.page_content, "question": question}
#         ).content
#         for doc in documents
#     )

#조금 더 직관적인 함수
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    results = []

    for doc in documents:
        result = map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        results.append(result)
    results = "\n\n".join(results)
    return results

 


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("홍길동을 묘사해줘!")


# embedder = OpenAIEmbeddings()

# #vector = embedder.embed_query("Hi! yoon. How are you?")
# vector = embedder.embed_documents([
#     "hi",
#     "how",
#     "are",
#     "you longer sentenses because"
# ])

# print(len(vector),len(vector[0]))

Created a chunk of size 935, which is longer than the specified 600


AIMessage(content='죄송합니다, 그 정보를 알 수 없습니다.')

In [9]:
result = vectorstore.similarity_search("홍길동은 어머니는 어때?")
print(result)

[Document(page_content='이루지 못했다. 하루는 어머니 침소에 가 길동이 울면서 말했다.\n홍길동전\uf07c허균\n“소자, 어머님과 더불어 전생의 연분이 귀중하여 금세(今世)에 모자지간(母子之間)이\n되었으니 은혜가 망극하옵니다. 그러나 소자의 팔자가 기박(奇薄)하여 천한 몸이 되오니\n품은 한이 깊사옵니다. 장부가 세상에 살면서 남의 천대를 받는 것이 당치 않은 일이라,\n소자가 자연히 기운을 억제하지 못하여 어머님 슬하를 떠나려 하옵니다. 엎드려\n바라오니, 어머님은 소자를 염려하지 마시고 귀한 몸을 보중하옵소서.”', metadata={'source': '../data/hong.pdf'}), Document(page_content='이루지 못했다. 하루는 어머니 침소에 가 길동이 울면서 말했다.\n홍길동전\uf07c허균\n“소자, 어머님과 더불어 전생의 연분이 귀중하여 금세(今世)에 모자지간(母子之間)이\n되었으니 은혜가 망극하옵니다. 그러나 소자의 팔자가 기박(奇薄)하여 천한 몸이 되오니\n품은 한이 깊사옵니다. 장부가 세상에 살면서 남의 천대를 받는 것이 당치 않은 일이라,\n소자가 자연히 기운을 억제하지 못하여 어머님 슬하를 떠나려 하옵니다. 엎드려\n바라오니, 어머님은 소자를 염려하지 마시고 귀한 몸을 보중하옵소서.”', metadata={'source': '../data/hong.pdf'}), Document(page_content='길동이 두 번 절하고 하직하니, 공이 붙들지 못하고 다만 무사하기만을 당부했다.\n길동은 또 어머니 침소에 가 이별을 고했다.\n“소자가 지금 슬하를 떠나고자 하옵니다. 다시 모실 날이 있을 것이오니 어머님은\n그사이 귀중한 몸을 보중하소서.”\n춘섬이 이 말을 듣고 무슨 변괴가 있었음을 짐작하여, 하직하는 아들을 보고 손을\n잡고 통곡하며 말했다.\n“네 어디로 향하고자 하느냐? 한집에 있어도 처소가 너무 멀어 매양 그리워하더니,\n이제 너를 정처 없이 보내고 어찌 살겠느냐? 너는 쉬이 돌아와 